In [3]:
from pathlib import Path
import pandas as pd, re

BASE_DIR = Path.cwd().parent
DATA_DIR = Path(BASE_DIR / "data")
CM_PATH  = DATA_DIR / "Cross_Map.csv"
PM_PATH  = DATA_DIR / "part_master.csv"
BACKUP   = DATA_DIR / "part_master.backup.csv"

IK_KEYS = ["ik_code", "ik", "iksan", "ik_part_type", "iksan_code"]
OK_KEYS = ["ok_code", "ok", "okcheon", "ok_part_type", "okcheon_code"]

def pick_col(df, candidates):
    low = {c.lower(): c for c in df.columns}
    for k in candidates:
        if k.lower() in low:
            return low[k.lower()]
    return None

cm = pd.read_csv(CM_PATH, dtype=str).fillna("")
display(cm.head())
print("cols:", list(cm.columns))

,category,ik_part_type,ok_part_type,remark
0,나사류,V111,2655,십자홈냄비작은나사
1,나사류,V112,2657,십자홈접시작은나사
2,나사류,V113,2658,십자홈바인드작은나사
3,나사류,V121,26022,십자홈냄비작은나사(S)
4,나사류,V122,26023,십자홈냄비작은나사(SP)


cols: ['category', 'ik_part_type', 'ok_part_type', 'remark']


In [35]:
def derive_part_type_ik(code):
    if not isinstance(code, str): return ""
    m = re.match(r"^(V\d{3})", code.strip().upper())
    return m.group(1) if m else code.strip().upper()

def derive_part_type_ok(code):
    if not isinstance(code, str): return ""
    s = code.strip().upper()
    m = re.match(r"^(\d{5})-\d{5}$", s)  # 필요시 변경
    return m.group(1) if m else s

assert "category" in cm.columns, "Cross_Map.csv에 'category' 컬럼 필요"
if "remark" not in cm.columns:
    cm["remark"] = ""

ik_col = pick_col(cm, IK_KEYS)
ok_col = pick_col(cm, OK_KEYS)
assert ik_col and ok_col, (ik_col, ok_col)

# IK
ik_rows = cm[cm[ik_col].astype(str).str.strip() != ""].copy()
ik_rows["site"] = "IKSAN"
ik_rows["part_type"] = ik_rows[ik_col].str.strip().str.upper()
ik_rows["part_type"] = ik_rows["part_type"].apply(derive_part_type_ik)
ik_rows["category"] = ik_rows["category"].str.strip()
#ik_rows["part_name"] = ""
ik_rows["desc"] = ik_rows["remark"]

# OK
ok_rows = cm[cm[ok_col].astype(str).str.strip() != ""].copy()
ok_rows["site"] = "OKCHEON"
ok_rows["part_type"] = ok_rows[ok_col].str.strip().str.upper()
ok_rows["part_type"] = ok_rows["part_type"].apply(derive_part_type_ok)
ok_rows["category"] = ok_rows["category"].str.strip()
#ok_rows["part_name"] = ""
ok_rows["desc"] = ok_rows["remark"]

pm_cols = ["site","part_type","desc","category"]
pm = pd.concat([ik_rows[pm_cols], ok_rows[pm_cols]], ignore_index=True)\
       .drop_duplicates(subset=["site","part_type"], keep="last")

display(pm.head(10))
print("rows:", len(pm))

,site,part_type,desc,category
0,IKSAN,V111,십자홈냄비작은나사,나사류
1,IKSAN,V112,십자홈접시작은나사,나사류
2,IKSAN,V113,십자홈바인드작은나사,나사류
3,IKSAN,V121,십자홈냄비작은나사(S),나사류
4,IKSAN,V122,십자홈냄비작은나사(SP),나사류
5,IKSAN,V123,십자홈냄비작은나사,나사류
6,IKSAN,V124,십자홈바인드작은나사(SP),나사류
7,IKSAN,V130,접시머리드릴링팁나사(뾰쪽끝),나사류
8,IKSAN,V131,접시머리드릴링팁나사(드릴끝),나사류
9,IKSAN,V132,등근접시머리드릴링팁나사(뾰쪽끝),나사류


rows: 394


In [37]:
if PM_PATH.exists():
    PM_PATH.replace(BACKUP)
    print("backup ->", BACKUP)

pm.to_csv(PM_PATH, index=False, encoding="cp949")
print("written ->", PM_PATH)

backup -> C:\Users\Allen\Desktop\Project\TYM\Code-Bridge-System\data\part_master.backup.csv
written -> C:\Users\Allen\Desktop\Project\TYM\Code-Bridge-System\data\part_master.csv
